In [3]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 76.0 MB/s eta 0:00:00


In [5]:
import pandas as pd
import geopandas as gpd
import sys
# sys.path.append(r'C:\Users\mfpen\Documents\Repositorios\geoloc2\Detecciondeterrenos')
# from codigos import obtener_curt

In [ ]:
def obtener_curt(data:gpd.GeoDataFrame, geom_col:str):
    '''
    (Function)
        Función que recibe un gdf y devuelve el mismo geodata con la clave unica de registro territorial
        en formato latitud(10 dígitos) + longitud(10 dígitos)
    (Parameters)
        - data    : GeoDataFrame al cual se le obtendrán las curt
        - geom_col: Str del nombre de la columna la cual contiene los polígonos 
    
    '''
    def convert_to_dms(coord):
        '''
    (Function)
        Funcion interna que recibe una coordenada y devuelve la misma pero en formato
        str de la forma: grados + minutos + segundos + diezmilesimas de segundo
    (Parameters)
        - coord: Coordenadas del centroide     
    '''
        degrees = int(coord)
        minutes = int((coord - degrees) * 60)
        seconds = int(((coord - degrees) * 60 - minutes) * 60)
        milliseconds = int((((coord - degrees) * 60 - minutes) * 60 - seconds) * 10000)
        return str(abs(degrees)).zfill(2)+str(abs(minutes)).zfill(2)+str(abs(seconds)).zfill(2)+str(abs(milliseconds)).zfill(4)

    data['centroid'] = data[geom_col].to_crs(4326).centroid
    data['lat'] = data['centroid'].y
    data['lon'] = data['centroid'].x
    data['CURT_f'] = data['lat'].apply(convert_to_dms)+data['lon'].apply(convert_to_dms)
    return data.drop(columns=['centroid','lat','lon'])

In [ ]:
CURT=gpd.read_file(r"D:\Secretaria\Bases_shp_INEGI\CURT\024 Atlacomulco\ATLACOMULCO_PREDIOS.shp")
CURT

,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry
0,15,024,0000,0240144801000000,194744919109952299131,NaN,"POLYGON ((-99.87493 19.79576, -99.87494 19.795..."
1,15,024,0000,0240109128000000,194737177109953100419,NaN,"POLYGON ((-99.88603 19.79363, -99.88612 19.793..."
2,15,024,0000,0240109116000000,194736032409953102406,NaN,"POLYGON ((-99.88603 19.79363, -99.88590 19.793..."
3,15,024,0000,0240142005000000,194824758809952102202,NaN,"POLYGON ((-99.86944 19.80691, -99.86948 19.806..."
4,15,024,0000,0240144837000000,194744232409952299340,NaN,"POLYGON ((-99.87493 19.79576, -99.87485 19.795..."
...,...,...,...,...,...,...,...
1919,15,024,0000,0240122320000000,194719536809952058052,NaN,"POLYGON ((-99.86814 19.78877, -99.86809 19.788..."
1920,15,024,0000,0240122321000000,194720252909952063672,NaN,"POLYGON ((-99.86820 19.78889, -99.86814 19.788..."
1921,15,024,0000,0240122328000000,194720252909952058206,NaN,"POLYGON ((-99.86827 19.78906, -99.86820 19.788..."
1922,15,024,0000,0240122345000000,194720239609952072994,NaN,"POLYGON ((-99.86859 19.78904, -99.86872 19.788..."


In [ ]:
casas_filtro=gpd.read_file(r"H:\.shortcut-targets-by-id\19j98C-PJkrk3gkgU7V03dLiJ4Gn-hnNe\geoshaps\Atlacomulco\Rotacion_Atlacomulco\prueba_deteccion_3_rotacion.shp")

In [ ]:
casas_filtro["ID_casas"]=casas_filtro.index
casas_filtro

,clase_dete,conf,area,status,geometry,ID_casas
0,establecimiento,0.70136,511.425330,correcto,"POLYGON ((-11128888.95573 2257450.58082, -1112...",0
1,casas,0.82655,182.508647,correcto,"POLYGON ((-11128894.36396 2257458.17207, -1112...",1
2,establecimiento,0.55637,322.899914,correcto,"POLYGON ((-11128702.18241 2256321.84985, -1112...",2
3,establecimiento,0.55807,465.296771,correcto,"POLYGON ((-11128709.79818 2256311.39002, -1112...",3
4,establecimiento,0.65544,187.522621,correcto,"POLYGON ((-11128695.31023 2256334.84175, -1112...",4
...,...,...,...,...,...,...
58498,establecimiento,0.38781,1073.993192,correcto,"POLYGON ((-11100399.98801 2243752.76801, -1110...",58498
58499,terreno_baldio,0.53138,361.006115,correcto,"POLYGON ((-11099977.25028 2245867.52183, -1109...",58499
58500,establecimiento,0.62800,210.586900,correcto,"POLYGON ((-11099543.48043 2242089.02879, -1109...",58500
58501,establecimiento,0.24212,195.544979,correcto,"POLYGON ((-11099434.89340 2243549.85882, -1109...",58501


In [ ]:
# CURT.geometry.value_counts()
casas_filtro.geometry.value_counts()

POLYGON ((-11128888.956 2257450.581, -11128884...    1
POLYGON ((-11128498.924 2255981.554, -11128488...    1
POLYGON ((-11128547.188 2255927.350, -11128543...    1
POLYGON ((-11128571.602 2255995.851, -11128569...    1
POLYGON ((-11128702.182 2256321.850, -11128697...    1
                                                    ..
POLYGON ((-11100394.036 2243756.817, -11100369...    1
POLYGON ((-11100399.988 2243752.768, -11100383...    1
POLYGON ((-11099977.250 2245867.522, -11099959...    1
POLYGON ((-11099543.480 2242089.029, -11099532...    1
POLYGON ((-11099425.118 2242077.771, -11099392...    1
Name: geometry, Length: 58503, dtype: int64

In [ ]:
#Se revisa el CRS de cada base para poder hacer el cruce entre ellas
print("crs CURT: ",CURT.crs)
print("crs casas: ",casas_filtro.crs)

crs CURT:  EPSG:4326
crs casas:  LOCAL_CS["WGS 84 / Pseudo-Mercator",LOCAL_DATUM["Unknown engineering datum",32767],UNIT["Meter",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]


In [ ]:
CURT=CURT.set_crs(6364,allow_override=True)
# CURT.geometry.head()
CURT=CURT.to_crs(3857)
CURT.geometry

0       POLYGON ((-11118026.626 2248851.074, -11118027...
1       POLYGON ((-11119261.940 2248599.490, -11119271...
2       POLYGON ((-11119261.940 2248599.490, -11119247...
3       POLYGON ((-11117414.936 2250171.302, -11117419...
4       POLYGON ((-11118026.626 2248851.074, -11118017...
                              ...                        
1919    POLYGON ((-11117270.073 2248025.083, -11117265...
1920    POLYGON ((-11117276.877 2248038.361, -11117270...
1921    POLYGON ((-11117284.711 2248059.038, -11117276...
1922    POLYGON ((-11117320.510 2248056.753, -11117335...
1923    POLYGON ((-11117291.290 2248079.719, -11117288...
Name: geometry, Length: 1924, dtype: geometry

In [ ]:
casas_filtro=casas_filtro.set_crs(3857,allow_override=True)
# casas_filtro.crs
# # CURT.geometry.head()
# casas_filtro.to_crs(3857)
casas_filtro.geometry

0        POLYGON ((-11128888.956 2257450.581, -11128884...
1        POLYGON ((-11128894.364 2257458.172, -11128890...
2        POLYGON ((-11128702.182 2256321.850, -11128697...
3        POLYGON ((-11128709.798 2256311.390, -11128704...
4        POLYGON ((-11128695.310 2256334.842, -11128692...
                               ...                        
58498    POLYGON ((-11100399.988 2243752.768, -11100383...
58499    POLYGON ((-11099977.250 2245867.522, -11099959...
58500    POLYGON ((-11099543.480 2242089.029, -11099532...
58501    POLYGON ((-11099434.893 2243549.859, -11099430...
58502    POLYGON ((-11099425.118 2242077.771, -11099392...
Name: geometry, Length: 58503, dtype: geometry

In [ ]:
casas_filtro=obtener_curt(casas_filtro,'geometry')
casas_filtro

C:\Users\mfpen\Documents\Repositorios\geoloc2\Detecciondeterrenos\codigos.py:347: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  data['centroid'] = data[geom_col].to_crs(4326).centroid


,clase_dete,conf,area,status,geometry,ID_casas,CURT_f
0,establecimiento,0.70136,511.425330,correcto,"POLYGON ((-11128888.956 2257450.581, -11128884...",0,19520650389958203663
1,casas,0.82655,182.508647,correcto,"POLYGON ((-11128894.364 2257458.172, -11128890...",1,19520678009958209589
2,establecimiento,0.55637,322.899914,correcto,"POLYGON ((-11128702.182 2256321.850, -11128697...",2,19513214269958144775
3,establecimiento,0.55807,465.296771,correcto,"POLYGON ((-11128709.798 2256311.390, -11128704...",3,19513178869958145855
4,establecimiento,0.65544,187.522621,correcto,"POLYGON ((-11128695.310 2256334.842, -11128692...",4,19513244849958143840
...,...,...,...,...,...,...,...
58498,establecimiento,0.38781,1073.993192,correcto,"POLYGON ((-11100399.988 2243752.768, -11100383...",58498,19450920559942586858
58499,terreno_baldio,0.53138,361.006115,correcto,"POLYGON ((-11099977.250 2245867.522, -11099959...",58499,19461399749942455637
58500,establecimiento,0.62800,210.586900,correcto,"POLYGON ((-11099543.480 2242089.029, -11099532...",58500,19441895799942316247
58501,establecimiento,0.24212,195.544979,correcto,"POLYGON ((-11099434.893 2243549.859, -11099430...",58501,19450350719942281620


In [ ]:
CURT.shape, casas_filtro.shape

((1924, 7), (58503, 7))

In [ ]:
cruce_curt=gpd.sjoin(CURT,casas_filtro)
cruce_curt

,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry,index_right,clase_dete,conf,area,status,ID_casas,CURT_f
0,15,024,0000,0240144801000000,194744919109952299131,NaN,"POLYGON ((-11118026.626 2248851.074, -11118027...",31445,casas,0.35628,561.565068,correcto,31445,19474431089952303057
4,15,024,0000,0240144837000000,194744232409952299340,NaN,"POLYGON ((-11118026.626 2248851.074, -11118017...",31445,casas,0.35628,561.565068,correcto,31445,19474431089952303057
0,15,024,0000,0240144801000000,194744919109952299131,NaN,"POLYGON ((-11118026.626 2248851.074, -11118027...",32129,establecimiento,0.45533,361.006115,correcto,32129,19474441869952297796
4,15,024,0000,0240144837000000,194744232409952299340,NaN,"POLYGON ((-11118026.626 2248851.074, -11118017...",32129,establecimiento,0.45533,361.006115,correcto,32129,19474441869952297796
0,15,024,0000,0240144801000000,194744919109952299131,NaN,"POLYGON ((-11118026.626 2248851.074, -11118027...",32130,establecimiento,0.45707,140.391267,correcto,32130,19474460699952299548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1897,15,024,0000,0240147019000000,194838722209951267616,NaN,"POLYGON ((-11116090.129 2250610.380, -11116091...",41159,casas,0.37332,140.391267,correcto,41159,19483871169951267208
1900,15,024,0000,0240144236000000,194827684709952076215,NaN,"POLYGON ((-11117282.590 2250268.896, -11117312...",36387,casas,0.31278,877.445419,correcto,36387,19482696059952067821
1900,15,024,0000,0240144236000000,194827684709952076215,NaN,"POLYGON ((-11117282.590 2250268.896, -11117312...",36389,establecimiento,0.35705,375.045242,correcto,36389,19482707239952066180
1906,15,024,0000,0240214105000000,194717131209950579254,NaN,"POLYGON ((-11115184.555 2247934.957, -11115187...",43389,casas,0.28564,342.955809,correcto,43389,19471689709950580613


In [ ]:
cruce_curt['CURT_si']=1
cruce_curt.tail(3)

,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry,index_right,clase_dete,conf,area,status,ID_casas,CURT_f,CURT_si
1900,15,024,0000,0240144236000000,194827684709952076215,NaN,"POLYGON ((-11117282.590 2250268.896, -11117312...",36389,establecimiento,0.35705,375.045242,correcto,36389,19482707239952066180,1
1906,15,024,0000,0240214105000000,194717131209950579254,NaN,"POLYGON ((-11115184.555 2247934.957, -11115187...",43389,casas,0.28564,342.955809,correcto,43389,19471689709950580613,1
1906,15,024,0000,0240214105000000,194717131209950579254,NaN,"POLYGON ((-11115184.555 2247934.957, -11115187...",43399,casas,0.73559,210.586900,correcto,43399,19471722329950575366,1


In [ ]:
cruce_curt.geometry.unique()

<GeometryArray>
[<POLYGON ((-11118026.626 2248851.074, -11118027.811 2248847.925, -11118039.2...>,
 <POLYGON ((-11118026.626 2248851.074, -11118017.083 2248848.289, -11118019.7...>,
 <POLYGON ((-11119261.94 2248599.49, -11119271.573 2248591.083, -11119282.706...>,
 <POLYGON ((-11119261.94 2248599.49, -11119247.839 2248579.669, -11119280.365...>,
 <POLYGON ((-11105996.084 2248177.528, -11105990.776 2248177.298, -11105975.9...>,
 <POLYGON ((-11106002.885 2248164.983, -11105976.342 2248163.832, -11105976.9...>,
 <POLYGON ((-11105995.653 2248190.349, -11105996.084 2248177.528, -11106002.4...>,
 <POLYGON ((-11105990.345 2248190.119, -11105990.776 2248177.298, -11105996.0...>,
 <POLYGON ((-11105975.911 2248176.653, -11105990.776 2248177.298, -11105990.3...>,
 <POLYGON ((-11106150.573 2247869.14, -11106159.393 2247869.155, -11106161.66...>,
 ...
 <POLYGON ((-11115075.635 2250634.278, -11115082.615 2250617.513, -11115112.4...>,
 <POLYGON ((-11115082.615 2250617.513, -11115075.635 2250634.278, 

In [ ]:
no_cruzanxcurt=CURT[~(CURT.geometry.isin(cruce_curt['geometry']))]
no_cruzanxcurt

,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry
3,15,024,0000,0240142005000000,194824758809952102202,NaN,"POLYGON ((-11117414.936 2250171.302, -11117419..."
5,15,024,0000,0240602413000000,195117681409955222284,NaN,"POLYGON ((-11123301.554 2255869.972, -11123312..."
10,15,024,0000,0241102980000000,194714662109946182809,NaN,"POLYGON ((-11106517.757 2247856.374, -11106558..."
11,15,024,0000,0241102978000000,194714818709946169924,NaN,"POLYGON ((-11106479.806 2247861.221, -11106517..."
96,15,024,0000,0241511115000000,194835629609949381160,NaN,"POLYGON ((-11112815.419 2250567.527, -11112691..."
...,...,...,...,...,...,...,...
1899,15,024,0000,0240147021000000,194840887209951271542,NaN,"POLYGON ((-11116081.256 2250713.011, -11116080..."
1901,15,024,0000,0240147022000000,194842015309951287433,NaN,"POLYGON ((-11116133.716 2250715.645, -11116144..."
1903,15,024,0000,0240147024000000,194842107009951266702,NaN,"POLYGON ((-11116069.050 2250713.645, -11116081..."
1904,15,024,0000,0240147006000000,194840706309951248676,NaN,"POLYGON ((-11115973.256 2250680.501, -11116000..."


In [ ]:
no_cruzanxcurt['CURT_si']=0
no_cruzanxcurt

c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry,CURT_si
3,15,024,0000,0240142005000000,194824758809952102202,NaN,"POLYGON ((-11117414.936 2250171.302, -11117419...",0
5,15,024,0000,0240602413000000,195117681409955222284,NaN,"POLYGON ((-11123301.554 2255869.972, -11123312...",0
10,15,024,0000,0241102980000000,194714662109946182809,NaN,"POLYGON ((-11106517.757 2247856.374, -11106558...",0
11,15,024,0000,0241102978000000,194714818709946169924,NaN,"POLYGON ((-11106479.806 2247861.221, -11106517...",0
96,15,024,0000,0241511115000000,194835629609949381160,NaN,"POLYGON ((-11112815.419 2250567.527, -11112691...",0
...,...,...,...,...,...,...,...,...
1899,15,024,0000,0240147021000000,194840887209951271542,NaN,"POLYGON ((-11116081.256 2250713.011, -11116080...",0
1901,15,024,0000,0240147022000000,194842015309951287433,NaN,"POLYGON ((-11116133.716 2250715.645, -11116144...",0
1903,15,024,0000,0240147024000000,194842107009951266702,NaN,"POLYGON ((-11116069.050 2250713.645, -11116081...",0
1904,15,024,0000,0240147006000000,194840706309951248676,NaN,"POLYGON ((-11115973.256 2250680.501, -11116000...",0


In [ ]:
cruce_curt.shape, no_cruzanxcurt.shape

((6839, 15), (183, 8))

In [ ]:
cruce_final_curt=pd.concat([cruce_curt,no_cruzanxcurt])
cruce_final_curt

,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry,index_right,clase_dete,conf,area,status,ID_casas,CURT_f,CURT_si
0,15,024,0000,0240144801000000,194744919109952299131,NaN,"POLYGON ((-11118026.626 2248851.074, -11118027...",31445.0,casas,0.35628,561.565068,correcto,31445.0,19474431089952303057,1
4,15,024,0000,0240144837000000,194744232409952299340,NaN,"POLYGON ((-11118026.626 2248851.074, -11118017...",31445.0,casas,0.35628,561.565068,correcto,31445.0,19474431089952303057,1
0,15,024,0000,0240144801000000,194744919109952299131,NaN,"POLYGON ((-11118026.626 2248851.074, -11118027...",32129.0,establecimiento,0.45533,361.006115,correcto,32129.0,19474441869952297796,1
4,15,024,0000,0240144837000000,194744232409952299340,NaN,"POLYGON ((-11118026.626 2248851.074, -11118017...",32129.0,establecimiento,0.45533,361.006115,correcto,32129.0,19474441869952297796,1
0,15,024,0000,0240144801000000,194744919109952299131,NaN,"POLYGON ((-11118026.626 2248851.074, -11118027...",32130.0,establecimiento,0.45707,140.391267,correcto,32130.0,19474460699952299548,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899,15,024,0000,0240147021000000,194840887209951271542,NaN,"POLYGON ((-11116081.256 2250713.011, -11116080...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1901,15,024,0000,0240147022000000,194842015309951287433,NaN,"POLYGON ((-11116133.716 2250715.645, -11116144...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1903,15,024,0000,0240147024000000,194842107009951266702,NaN,"POLYGON ((-11116069.050 2250713.645, -11116081...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1904,15,024,0000,0240147006000000,194840706309951248676,NaN,"POLYGON ((-11115973.256 2250680.501, -11116000...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
#Para detectar duplicados x geometría
geo_dupli=cruce_final_curt.geometry.value_counts()[cruce_final_curt.geometry.value_counts()>1].index
cruce_final_curt.loc[cruce_final_curt.geometry.isin(geo_dupli),"dupGEO"]=1
cruce_final_curt.loc[cruce_final_curt.dupGEO != 1, "dupGEO"]=0
cruce_final_curt


,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry,index_right,clase_dete,conf,area,status,ID_casas,CURT_f,CURT_si,dupGEO
0,15,024,0000,0240144801000000,194744919109952299131,NaN,"POLYGON ((-11118026.626 2248851.074, -11118027...",31445.0,casas,0.35628,561.565068,correcto,31445.0,19474431089952303057,1,1.0
4,15,024,0000,0240144837000000,194744232409952299340,NaN,"POLYGON ((-11118026.626 2248851.074, -11118017...",31445.0,casas,0.35628,561.565068,correcto,31445.0,19474431089952303057,1,1.0
0,15,024,0000,0240144801000000,194744919109952299131,NaN,"POLYGON ((-11118026.626 2248851.074, -11118027...",32129.0,establecimiento,0.45533,361.006115,correcto,32129.0,19474441869952297796,1,1.0
4,15,024,0000,0240144837000000,194744232409952299340,NaN,"POLYGON ((-11118026.626 2248851.074, -11118017...",32129.0,establecimiento,0.45533,361.006115,correcto,32129.0,19474441869952297796,1,1.0
0,15,024,0000,0240144801000000,194744919109952299131,NaN,"POLYGON ((-11118026.626 2248851.074, -11118027...",32130.0,establecimiento,0.45707,140.391267,correcto,32130.0,19474460699952299548,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899,15,024,0000,0240147021000000,194840887209951271542,NaN,"POLYGON ((-11116081.256 2250713.011, -11116080...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0
1901,15,024,0000,0240147022000000,194842015309951287433,NaN,"POLYGON ((-11116133.716 2250715.645, -11116144...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0
1903,15,024,0000,0240147024000000,194842107009951266702,NaN,"POLYGON ((-11116069.050 2250713.645, -11116081...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0
1904,15,024,0000,0240147006000000,194840706309951248676,NaN,"POLYGON ((-11115973.256 2250680.501, -11116000...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0


In [ ]:
#Para detectar duplicados x curt
curt_dupli=cruce_final_curt.curt.value_counts()[cruce_final_curt.curt.value_counts()>1].index
cruce_final_curt.loc[cruce_final_curt.curt.isin(curt_dupli),"dupCURT"]=1
cruce_final_curt.loc[cruce_final_curt.dupCURT != 1, "dupCURT"]=0
cruce_final_curt

,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry,index_right,clase_dete,conf,area,status,ID_casas,CURT_f,CURT_si,dupGEO,dupCURT
0,15,024,0000,0240144801000000,194744919109952299131,NaN,"POLYGON ((-11118026.626 2248851.074, -11118027...",31445.0,casas,0.35628,561.565068,correcto,31445.0,19474431089952303057,1,1.0,1.0
4,15,024,0000,0240144837000000,194744232409952299340,NaN,"POLYGON ((-11118026.626 2248851.074, -11118017...",31445.0,casas,0.35628,561.565068,correcto,31445.0,19474431089952303057,1,1.0,1.0
0,15,024,0000,0240144801000000,194744919109952299131,NaN,"POLYGON ((-11118026.626 2248851.074, -11118027...",32129.0,establecimiento,0.45533,361.006115,correcto,32129.0,19474441869952297796,1,1.0,1.0
4,15,024,0000,0240144837000000,194744232409952299340,NaN,"POLYGON ((-11118026.626 2248851.074, -11118017...",32129.0,establecimiento,0.45533,361.006115,correcto,32129.0,19474441869952297796,1,1.0,1.0
0,15,024,0000,0240144801000000,194744919109952299131,NaN,"POLYGON ((-11118026.626 2248851.074, -11118027...",32130.0,establecimiento,0.45707,140.391267,correcto,32130.0,19474460699952299548,1,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899,15,024,0000,0240147021000000,194840887209951271542,NaN,"POLYGON ((-11116081.256 2250713.011, -11116080...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0
1901,15,024,0000,0240147022000000,194842015309951287433,NaN,"POLYGON ((-11116133.716 2250715.645, -11116144...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0
1903,15,024,0000,0240147024000000,194842107009951266702,NaN,"POLYGON ((-11116069.050 2250713.645, -11116081...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0
1904,15,024,0000,0240147006000000,194840706309951248676,NaN,"POLYGON ((-11115973.256 2250680.501, -11116000...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0


In [ ]:
casas_nocruzan = casas_filtro[~casas_filtro.ID_casas.isin(cruce_curt.ID_casas.unique())]
casas_nocruzan['YOLO_si'] = 0
casas_nocruzan.tail(2)

c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,clase_dete,conf,area,status,geometry,ID_casas,CURT_f,YOLO_si
58501,establecimiento,0.24212,195.544979,correcto,"POLYGON ((-11099434.893 2243549.859, -11099430...",58501,19450350719942281620,0
58502,establecimiento,0.27316,649.811007,correcto,"POLYGON ((-11099425.118 2242077.771, -11099392...",58502,19441838869942275749,0


In [ ]:
cruce_final_curt.shape, casas_nocruzan.shape

((7022, 17), (54371, 8))

In [ ]:
cruce_final_curt.columns, casas_nocruzan.columns

(Index(['cve_ent', 'cve_mun', 'cve_loc', 'id_cat', 'curt', 'notas', 'geometry',
        'index_right', 'clase_dete', 'conf', 'area', 'status', 'ID_casas',
        'CURT_f', 'CURT_si', 'dupGEO', 'dupCURT'],
       dtype='object'),
 Index(['clase_dete', 'conf', 'area', 'status', 'geometry', 'ID_casas',
        'CURT_f', 'YOLO_si'],
       dtype='object'))

In [ ]:
cruce_final =pd.concat([cruce_final_curt,casas_nocruzan])
cruce_final

,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry,index_right,clase_dete,conf,area,status,ID_casas,CURT_f,CURT_si,dupGEO,dupCURT,YOLO_si
0,15,024,0000,0240144801000000,194744919109952299131,NaN,"POLYGON ((-11118026.626 2248851.074, -11118027...",31445.0,casas,0.35628,561.565068,correcto,31445.0,19474431089952303057,1.0,1.0,1.0,NaN
4,15,024,0000,0240144837000000,194744232409952299340,NaN,"POLYGON ((-11118026.626 2248851.074, -11118017...",31445.0,casas,0.35628,561.565068,correcto,31445.0,19474431089952303057,1.0,1.0,1.0,NaN
0,15,024,0000,0240144801000000,194744919109952299131,NaN,"POLYGON ((-11118026.626 2248851.074, -11118027...",32129.0,establecimiento,0.45533,361.006115,correcto,32129.0,19474441869952297796,1.0,1.0,1.0,NaN
4,15,024,0000,0240144837000000,194744232409952299340,NaN,"POLYGON ((-11118026.626 2248851.074, -11118017...",32129.0,establecimiento,0.45533,361.006115,correcto,32129.0,19474441869952297796,1.0,1.0,1.0,NaN
0,15,024,0000,0240144801000000,194744919109952299131,NaN,"POLYGON ((-11118026.626 2248851.074, -11118027...",32130.0,establecimiento,0.45707,140.391267,correcto,32130.0,19474460699952299548,1.0,1.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58498,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-11100399.988 2243752.768, -11100383...",NaN,establecimiento,0.38781,1073.993192,correcto,58498.0,19450920559942586858,NaN,NaN,NaN,0.0
58499,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-11099977.250 2245867.522, -11099959...",NaN,terreno_baldio,0.53138,361.006115,correcto,58499.0,19461399749942455637,NaN,NaN,NaN,0.0
58500,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-11099543.480 2242089.029, -11099532...",NaN,establecimiento,0.62800,210.586900,correcto,58500.0,19441895799942316247,NaN,NaN,NaN,0.0
58501,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-11099434.893 2243549.859, -11099430...",NaN,establecimiento,0.24212,195.544979,correcto,58501.0,19450350719942281620,NaN,NaN,NaN,0.0


In [ ]:
cruce_final.loc[cruce_final.YOLO_si.isna(),"YOLO_si"]=1

In [ ]:
cruce_final.rename(columns={'index_right':'ID_curt'},inplace=True)

In [ ]:
cruce_final.columns

Index(['cve_ent', 'cve_mun', 'cve_loc', 'id_cat', 'curt', 'notas', 'geometry',
       'ID_curt', 'clase_dete', 'conf', 'area', 'status', 'ID_casas', 'CURT_f',
       'CURT_si', 'dupGEO', 'dupCURT', 'YOLO_si'],
      dtype='object')

In [ ]:
cruce_final.to_file(r"D:\Secretaria\cruces_bases\limpiezas_shp_base\Atlacomulco/Atlacomulco_casasxcurt.shp")

Cruce casas x industrias

In [ ]:
path_industrial = r"H:\.shortcut-targets-by-id\19j98C-PJkrk3gkgU7V03dLiJ4Gn-hnNe\geoshaps\Atlacomulco\industria\Atlaco_industrias_filtro.shp"
industrias = gpd.read_file(path_industrial)

In [ ]:
cruce_final.shape, industrias.shape

((61393, 18), (2086, 11))

In [ ]:
cruce_casasxind = gpd.sjoin(industrias,cruce_final)
cruce_casasxind

,puntos,zona,area_left,rectangulo,Medida_lx,Medida_ly,Max_xy,Min_xy,f_lado,f_area,...,clase_dete,conf,area_right,status,ID_casas,CURT_f,CURT_si,dupGEO,dupCURT,YOLO_si
1,307,02,1040.554719,1837.846646,40.6278,45.2363,45.2363,40.6278,1.113432,0.566181,...,establecimiento,0.62966,304.849608,correcto,40669.0,19453695049951349593,1.0,1.0,1.0,1.0
1,307,02,1040.554719,1837.846646,40.6278,45.2363,45.2363,40.6278,1.113432,0.566181,...,establecimiento,0.77136,204.570132,correcto,40662.0,19453994009951335305,1.0,1.0,1.0,1.0
1,307,02,1040.554719,1837.846646,40.6278,45.2363,45.2363,40.6278,1.113432,0.566181,...,establecimiento,0.60915,610.702011,correcto,40668.0,19453684639951342368,1.0,1.0,1.0,1.0
1,307,02,1040.554719,1837.846646,40.6278,45.2363,45.2363,40.6278,1.113432,0.566181,...,establecimiento,0.53915,1147.197210,correcto,40665.0,19453745999951341696,1.0,1.0,1.0,1.0
1,307,02,1040.554719,1837.846646,40.6278,45.2363,45.2363,40.6278,1.113432,0.566181,...,establecimiento,0.21399,457.274412,correcto,40820.0,19453741119951319731,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084,127,05,312.112901,511.480173,15.4999,32.9990,32.9990,15.4999,2.128981,0.610215,...,establecimiento,0.27031,126.352140,correcto,4469.0,19521890679955410598,NaN,NaN,NaN,0.0
2084,127,05,312.112901,511.480173,15.4999,32.9990,32.9990,15.4999,2.128981,0.610215,...,casas,0.82385,1055.942886,correcto,4828.0,19521883029955401325,NaN,NaN,NaN,0.0
2084,127,05,312.112901,511.480173,15.4999,32.9990,32.9990,15.4999,2.128981,0.610215,...,establecimiento,0.64335,589.643321,correcto,4465.0,19521961289955406966,NaN,NaN,NaN,0.0
2085,91,05,352.611331,617.726055,25.5971,24.1348,25.5971,24.1348,1.060589,0.570822,...,establecimiento,0.72352,649.811007,correcto,7601.0,19522536769955192594,NaN,NaN,NaN,0.0


In [ ]:
cruce_casasxind['Ind_si']=1
cruce_casasxind

,puntos,zona,area_left,rectangulo,Medida_lx,Medida_ly,Max_xy,Min_xy,f_lado,f_area,...,conf,area_right,status,ID_casas,CURT_f,CURT_si,dupGEO,dupCURT,YOLO_si,Ind_si
1,307,02,1040.554719,1837.846646,40.6278,45.2363,45.2363,40.6278,1.113432,0.566181,...,0.62966,304.849608,correcto,40669.0,19453695049951349593,1.0,1.0,1.0,1.0,1
1,307,02,1040.554719,1837.846646,40.6278,45.2363,45.2363,40.6278,1.113432,0.566181,...,0.77136,204.570132,correcto,40662.0,19453994009951335305,1.0,1.0,1.0,1.0,1
1,307,02,1040.554719,1837.846646,40.6278,45.2363,45.2363,40.6278,1.113432,0.566181,...,0.60915,610.702011,correcto,40668.0,19453684639951342368,1.0,1.0,1.0,1.0,1
1,307,02,1040.554719,1837.846646,40.6278,45.2363,45.2363,40.6278,1.113432,0.566181,...,0.53915,1147.197210,correcto,40665.0,19453745999951341696,1.0,1.0,1.0,1.0,1
1,307,02,1040.554719,1837.846646,40.6278,45.2363,45.2363,40.6278,1.113432,0.566181,...,0.21399,457.274412,correcto,40820.0,19453741119951319731,1.0,1.0,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084,127,05,312.112901,511.480173,15.4999,32.9990,32.9990,15.4999,2.128981,0.610215,...,0.27031,126.352140,correcto,4469.0,19521890679955410598,NaN,NaN,NaN,0.0,1
2084,127,05,312.112901,511.480173,15.4999,32.9990,32.9990,15.4999,2.128981,0.610215,...,0.82385,1055.942886,correcto,4828.0,19521883029955401325,NaN,NaN,NaN,0.0,1
2084,127,05,312.112901,511.480173,15.4999,32.9990,32.9990,15.4999,2.128981,0.610215,...,0.64335,589.643321,correcto,4465.0,19521961289955406966,NaN,NaN,NaN,0.0,1
2085,91,05,352.611331,617.726055,25.5971,24.1348,25.5971,24.1348,1.060589,0.570822,...,0.72352,649.811007,correcto,7601.0,19522536769955192594,NaN,NaN,NaN,0.0,1


In [ ]:
no_cruzanxInd=industrias[~(industrias.geometry.isin(cruce_casasxind['geometry']))]
no_cruzanxInd.tail(2)

,puntos,zona,area,rectangulo,Medida_lx,Medida_ly,Max_xy,Min_xy,f_lado,f_area,geometry
2069,131,09,516.873390,991.496913,43.4081,22.8472,43.4081,22.8472,1.899931,0.521306,"POLYGON ((-11114801.391 2255514.595, -11114783..."
2073,159,09,392.248779,732.872718,23.7540,30.4506,30.4506,23.7540,1.281915,0.535221,"POLYGON ((-11113765.400 2256289.599, -11113753..."


In [ ]:
no_cruzanxInd['Ind_si']=0
no_cruzanxInd.tail(2)

c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,puntos,zona,area,rectangulo,Medida_lx,Medida_ly,Max_xy,Min_xy,f_lado,f_area,geometry,Ind_si
2069,131,09,516.873390,991.496913,43.4081,22.8472,43.4081,22.8472,1.899931,0.521306,"POLYGON ((-11114801.391 2255514.595, -11114783...",0
2073,159,09,392.248779,732.872718,23.7540,30.4506,30.4506,23.7540,1.281915,0.535221,"POLYGON ((-11113765.400 2256289.599, -11113753...",0


In [ ]:
cruce_casasxind.shape, no_cruzanxInd.shape

((11134, 30), (116, 12))

In [ ]:
crucexInd=pd.concat([cruce_casasxind,no_cruzanxInd])
crucexInd

,puntos,zona,area_left,rectangulo,Medida_lx,Medida_ly,Max_xy,Min_xy,f_lado,f_area,...,area_right,status,ID_casas,CURT_f,CURT_si,dupGEO,dupCURT,YOLO_si,Ind_si,area
1,307,02,1040.554719,1837.846646,40.6278,45.2363,45.2363,40.6278,1.113432,0.566181,...,304.849608,correcto,40669.0,19453695049951349593,1.0,1.0,1.0,1.0,1,NaN
1,307,02,1040.554719,1837.846646,40.6278,45.2363,45.2363,40.6278,1.113432,0.566181,...,204.570132,correcto,40662.0,19453994009951335305,1.0,1.0,1.0,1.0,1,NaN
1,307,02,1040.554719,1837.846646,40.6278,45.2363,45.2363,40.6278,1.113432,0.566181,...,610.702011,correcto,40668.0,19453684639951342368,1.0,1.0,1.0,1.0,1,NaN
1,307,02,1040.554719,1837.846646,40.6278,45.2363,45.2363,40.6278,1.113432,0.566181,...,1147.197210,correcto,40665.0,19453745999951341696,1.0,1.0,1.0,1.0,1,NaN
1,307,02,1040.554719,1837.846646,40.6278,45.2363,45.2363,40.6278,1.113432,0.566181,...,457.274412,correcto,40820.0,19453741119951319731,1.0,1.0,1.0,1.0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2063,63,09,NaN,474.998521,24.9998,19.0001,24.9998,19.0001,1.315772,0.791053,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,375.748830
2067,167,09,NaN,758.747637,18.1246,42.4997,42.4997,18.1246,2.344863,0.697694,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,529.373352
2068,187,09,NaN,3088.990381,80.4119,38.6390,80.4119,38.6390,2.081107,0.704799,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2177.118221
2069,131,09,NaN,991.496913,43.4081,22.8472,43.4081,22.8472,1.899931,0.521306,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,516.873390


In [ ]:
nocruzan_casas =cruce_final[~cruce_final.ID_casas.isin(cruce_casasxind.ID_casas.unique())]
nocruzan_casas['Induxcasa'] = 0
nocruzan_casas.tail(2)

c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry,ID_curt,clase_dete,conf,area,status,ID_casas,CURT_f,CURT_si,dupGEO,dupCURT,YOLO_si,Induxcasa
58501,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-11099434.893 2243549.859, -11099430...",NaN,establecimiento,0.24212,195.544979,correcto,58501.0,19450350719942281620,NaN,NaN,NaN,0.0,0
58502,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-11099425.118 2242077.771, -11099392...",NaN,establecimiento,0.27316,649.811007,correcto,58502.0,19441838869942275749,NaN,NaN,NaN,0.0,0


In [ ]:
cruce_final_ind =pd.concat([crucexInd,nocruzan_casas])
cruce_final_ind

,puntos,zona,area_left,rectangulo,Medida_lx,Medida_ly,Max_xy,Min_xy,f_lado,f_area,...,status,ID_casas,CURT_f,CURT_si,dupGEO,dupCURT,YOLO_si,Ind_si,area,Induxcasa
1,307.0,02,1040.554719,1837.846646,40.6278,45.2363,45.2363,40.6278,1.113432,0.566181,...,correcto,40669.0,19453695049951349593,1.0,1.0,1.0,1.0,1.0,NaN,NaN
1,307.0,02,1040.554719,1837.846646,40.6278,45.2363,45.2363,40.6278,1.113432,0.566181,...,correcto,40662.0,19453994009951335305,1.0,1.0,1.0,1.0,1.0,NaN,NaN
1,307.0,02,1040.554719,1837.846646,40.6278,45.2363,45.2363,40.6278,1.113432,0.566181,...,correcto,40668.0,19453684639951342368,1.0,1.0,1.0,1.0,1.0,NaN,NaN
1,307.0,02,1040.554719,1837.846646,40.6278,45.2363,45.2363,40.6278,1.113432,0.566181,...,correcto,40665.0,19453745999951341696,1.0,1.0,1.0,1.0,1.0,NaN,NaN
1,307.0,02,1040.554719,1837.846646,40.6278,45.2363,45.2363,40.6278,1.113432,0.566181,...,correcto,40820.0,19453741119951319731,1.0,1.0,1.0,1.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,correcto,58496.0,19452424219943078647,NaN,NaN,NaN,0.0,NaN,596.662885,0.0
58499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,correcto,58499.0,19461399749942455637,NaN,NaN,NaN,0.0,NaN,361.006115,0.0
58500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,correcto,58500.0,19441895799942316247,NaN,NaN,NaN,0.0,NaN,210.586900,0.0
58501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,correcto,58501.0,19450350719942281620,NaN,NaN,NaN,0.0,NaN,195.544979,0.0


In [ ]:
cruce_final_ind.rename(columns={"index_right":"ID_ind"},inplace=True)
cruce_final_ind.columns

Index(['puntos', 'zona', 'area_left', 'rectangulo', 'Medida_lx', 'Medida_ly',
       'Max_xy', 'Min_xy', 'f_lado', 'f_area', 'geometry', 'ID_ind', 'cve_ent',
       'cve_mun', 'cve_loc', 'id_cat', 'curt', 'notas', 'ID_curt',
       'clase_dete', 'conf', 'area_right', 'status', 'ID_casas', 'CURT_f',
       'CURT_si', 'dupGEO', 'dupCURT', 'YOLO_si', 'Ind_si', 'area',
       'Induxcasa'],
      dtype='object')

In [ ]:
cruce_final_ind.to_file(r"D:\Secretaria\cruces_bases\limpiezas_shp_base\Atlacomulco/Atlacomulco_casas_Indxcurt.shp")

In [ ]:
cruce_final_ind.read_file